In [6]:
import numpy as np
import csv
import sys
import json

# def erosion(pointcloud, neighbours, point_idx):
#     z_vals = [pointcloud[neighbour,-1] for neighbour in neighbours]
#     pointcloud[point_idx, -1] = min(z_vals)
#     return pointcloud

# def dilation(pointcloud, neighbours, point_idx):
#     z_vals = [pointcloud[neighbour,-1] for neighbour in neighbours]
#     pointcloud[point_idx, -1] = max(z_vals)
#     return pointcloud

# def boundbox_neighbours(point, pointcloud, window_size):

#     minx = point[0] - window_size/2.0
#     miny = point[1] - window_size/2.0
#     maxx = point[0] + window_size/2.0
#     maxy = point[1] + window_size/2.0

#     neighbour_inds = []
#     for point_idx, p in enumerate(pointcloud):
#         if (p[0]<=maxx and p[0]>=minx and p[1]<=maxy and p[1]>=miny):
#             neighbour_inds.append(point_idx)
#     return neighbour_inds

# def findNeighbours(point, pointcloud, window_size):
#     minx = point[0] - window_size/2.0
#     maxx = point[0] + window_size/2.0
    
#     miny = point[1] - window_size/2.0
#     maxy = point[1] + window_size/2.0
    
#     return pointcloud.flatMap( lambda pt: (True if (pt[0] <= maxx and pt[0] >= minx and pt[1] <= maxy and pt[1] >= miny) else False) ) 


points = sc.textFile('odm_mesh_small.xyz')
points_split = points.map(lambda x: (x.split(" ")))
points_parsed = points_split.map(lambda line: [float(line[0]), float(line[1]), float(line[2])])
cnt = points_parsed.take(5)
print cnt
print "\n"
points_enum = points_parsed.zipWithIndex().map(lambda x: (x[1],x[0]))
print points_enum.take(4)
print "\n"

##################################################################

#np_pointcloud = np.array(pointcloud)

# Build different windows and height thresholds
windows = []
height_thresholds = []

# Default parameters
slope = 1.0
cell_size = 1.0
base = 2.0
max_distance = 1
initial_distance = 0.5
max_window_size = 20
window_type = 'linear'

window_size = 0.0
i = 0
while window_size < max_window_size:
    # Create different windows
    if window_type == 'linear':
        window_size = cell_size * ((2*(i+1)*base) + 1)
    elif window_type == 'exponential':
        window_size = cell_size * ((2*base**(i+1) + 1))
    if window_size > max_window_size:
        break
    windows.append(window_size)
    # Create corresponding height threshold
    if i == 0:
        height_threshold = initial_distance
    elif height_threshold > max_distance:
        height_threshold = max_distance
    else:
        height_threshold = slope*(windows[-1]-windows[-2])*cell_size + initial_distance


    height_thresholds.append(height_threshold)
    i += 1

for window in windows:
    print 'window size: ', window

for thresh in height_thresholds:
    print 'height thresholds: ', thresh

    
    
######################## MAKE CARTESIAN (KIND OF MATRIX points by points - all possible combinations) ##################

points_matrix = points_enum.cartesian(points_enum)
neighbours = points_matrix.map(lambda (x,y): (x[0],y[0]) if ( y[1][0] >= x[1][0] - window_size/2 and y[1][0] <= x[1][0] + window_size/2 and y[1][1] >= x[1][1] - window_size/2 and y[1][1] <= x[1][1] + window_size/2 ) else None)
print neighbours.take(14)
print "\n"   

######################## THE TUPLES ARE PAIRS OF NEIGHBOURS ############################################################
# PORABABLY WE DON'T NEED TO SORT ANYTHINGS AS WE'LL ITERATE OVER COMPLETE LISTs ANYWAY


# ACUTALLY WE CAN JUST RETURN INDEX OF POINT x[0], INDEX OF NEIGHBOUR y[0] AND THE Z-VALUE OF ITS NEIGHBOUR Y[1][2]
neighbours = points_matrix.map(lambda (x,y): (x[0], y[0], y[1][2]) if ( y[1][0] >= x[1][0] - window_size/2 and y[1][0] <= x[1][0] + window_size/2 and y[1][1] >= x[1][1] - window_size/2 and y[1][1] <= x[1][1] + window_size/2 ) else None)
print neighbours.take(14)
print "\n"


# FIND MIN z_vals -> sort by z_value and pick the smallest 
erosion = neighbours.sortBy(lambda x: x[2])
print erosion.map(lambda x: x[2]).take(3)
# I DON'T HAVE MORE TIME FOR THIS TONIGHT BUT THERE IS SOME BUG HERE :)))) 


#         def erosion(pointcloud, neighbours, point_idx):
#             z_vals = [pointcloud[neighbour,-1] for neighbour in neighbours]
#             pointcloud[point_idx, -1] = min(z_vals)
#             return pointcloud

#         def dilation(pointcloud, neighbours, point_idx):
#             z_vals = [pointcloud[neighbour,-1] for neighbour in neighbours]
#             pointcloud[point_idx, -1] = max(z_vals)
#             return pointcloud
        


# flags = np.zeros(np_pointcloud.shape[0])
# pointcloud_copy = np.copy(np_pointcloud)


# SOMEHOW WE NEED TO PREVENT FINDING ALL THE NEIGHBOURS FROM SCRATCH - WE JUST NEED TO AUGMENT PREVIOUSLY FOUND SET

for window, thres in zip(windows, height_thresholds):

    # window = windows[-1]
    # thresh = height_thresholds[-1]

#     for point_idx, point in enumerate(pointcloud_copy):
#       neighbours = boundbox_neighbours(point, pointcloud_copy, window)
#         
#         if neighbours:
            # Open operator (erosion + dilation)
#             pointcloud_copy = erosion(pointcloud_copy, neighbours, point_idx)
#             pointcloud_copy = dilation(pointcloud_copy, neighbours, point_idx)
            # pointcloud_copy = Z
    

    for point_idx in range(np_pointcloud.shape[0]):
        if ( abs(np_pointcloud[point_idx,-1] - pointcloud_copy[point_idx,-1])) > thres:
            flags[point_idx] = 1


with open('dataset/pcloud1.xyz', 'wb') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=' ')
    csvwriter.writerows(np_pointcloud[np.where(flags == 1)[0], :].tolist())

with open('dataset/pcloud2.xyz', 'wb') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=' ')
    csvwriter.writerows(np_pointcloud[np.where(flags == 0)[0], :].tolist())




[[-255.509995, -55.193001, -154.910004], [-255.460007, -55.193001, -154.919998], [-264.429993, -55.193001, -154.699997], [-262.190002, -55.193001, -154.759995], [-259.950012, -55.193001, -154.850006]]


[(0, [-255.509995, -55.193001, -154.910004]), (1, [-255.460007, -55.193001, -154.919998]), (2, [-264.429993, -55.193001, -154.699997]), (3, [-262.190002, -55.193001, -154.759995])]


window size:  5.0
window size:  9.0
window size:  13.0
window size:  17.0
height thresholds:  0.5
height thresholds:  4.5
height thresholds:  1
height thresholds:  4.5
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), None, None, None]


[(0, 0, -154.910004), (0, 1, -154.919998), (0, 2, -154.699997), (0, 3, -154.759995), (0, 4, -154.850006), (0, 5, -154.669998), (0, 6, -155.110001), (0, 7, -154.910004), (0, 8, -154.710007), (0, 9, -154.490005), (0, 10, -154.240005), None, None, None]




Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 24, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 101, in main
    process()
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/worker.py", line 96, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/rdd.py", line 2252, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/rdd.py", line 2252, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/rdd.py", lin